In [1]:
import numpy as np
import pandas as pd
import json, os, sys

# SECTION TO CORRECT ISSUE I HAD WITH JSON FILES

## THIS IS NOW FIXED PLEASE JUST LOAD THE CLEAN FILES. CODE BELOW

In [2]:
bert_results = pd.read_json('results/bert_results_25august.json')
roberta_results = pd.read_json('results/roberta_results_25august.json')
t5_results = pd.read_json('results/t5_results_25august.json')

# NOTE

The results as-is from Colab have some issues due to the way I have constructed them. 

- Please CTRL-F and look for `"]["`. Replace with `"],["`
- Please insert an opening square bracket at start, and a closing one at the end. (enclose the whole contents into square brackets.)

These two steps fix the issues and can be read with `pd.read_json()`

In [3]:
final_dfs = []
for df in [bert_results, roberta_results]:
    all_vals = []
    for _, row in df.stack().to_frame().droplevel(1).iterrows():
        values = row.values
        for val in values:
            for item in val:
                if val is not None:
                    all_vals.append(item)
    final_dfs.append(all_vals)

for df in [t5_results]:
    all_vals = []
    for _, row in df.stack().to_frame().droplevel(1).iterrows():
        values = row.values
        all_vals.append(values[0])
    final_dfs.append(all_vals)

In [19]:
bert_results, roberta_results, t5_results = pd.DataFrame(
    final_dfs[0]), pd.DataFrame(final_dfs[1]), pd.DataFrame(final_dfs[2])

for df in [bert_results, roberta_results, t5_results]:
    df['processed_answer'] = df['answer'].apply(lambda x: str(x).lower())
    df['processed_pred'] = df['token_str'].apply(lambda x: str(x).lower().strip())
    df['correct'] = np.where(df['processed_answer'] == df['processed_pred'], 1, 0)

In [20]:
bert_results.to_json('results/bert_results_25august_clean.json')
roberta_results.to_json('results/roberta_results_25august_clean.json')
t5_results.to_json('results/t5_results_25august_clean.json')

# CLEAN VERSIONS

In [21]:
bert_results = pd.read_json('results/bert_results_25august_clean.json')
roberta_results = pd.read_json('results/roberta_results_25august_clean.json')
t5_results = pd.read_json('results/t5_results_25august_clean.json')

In [75]:
t5_results.groupby('ids').first()

,sequence,token_str,answer,processed_answer,correct,processed_pred
ids,,,,,,
0001b29ee9e518b252beaedd07f7193c4c5bd405,Total UK exports of services (excluding travel...,17.0%,14.3%,14.3%,0,17.0%
000aba0de1890391d98fbba20af8abe629a95452,"Between 9 March and 30 June 2020, prior to the...",61,"5,330","5,330",0,none
000cbe156681cb91437a5f0953c0047f83e47f35,"In January to February 2020, 26% of adults in ...",26%,18%,18%,0,26%
00139eecad6df8931ffea624f73a530b0a1acb78,"“While they were more optimistic, young people...",many,42%,42%,0,many
005aed79078fc309e9d95f917bcb9d15dbff6d76,5% of children living in a city reported “ofte...,5%,19.5%,19.5%,0,5%
...,...,...,...,...,...,...
ffdef2a0629d19756210d13e448248ba983eb355,38% of adults said they had run out of money t...,38% of adults saying they were,12%,12%,0,38% of adults saying they were
ffe1db00c7b8cea213ab00eb0f57d28f1cb6dcc0,"Similar to ,, Ischaemic heart disease was the ...",",",previous years,previous years,0,","
ffec045e37d60b9eea6efead436c1d5edfad275a,Total current healthcare expenditure in the UK...,2018,2018,2018,1,2018


In [25]:
bert_first_correct = bert_results.groupby('ids').first().correct.sum()
roberta_first_correct = roberta_results.groupby('ids').first().correct.sum()
t5_first_correct = t5_results.groupby('ids').first().correct.sum()

print(
    f"Bert results, first prediction correct: {bert_first_correct} out of 6070; ({bert_first_correct / 60.70})")
print(
    f"RoBERTa results, first prediction correct: {roberta_first_correct} out of 6070; ({roberta_first_correct/60.70}) ")
print(
    f"T5 results, first prediction correct: {t5_first_correct} out of 6070; ({t5_first_correct/60.70})")

Bert results, first prediction correct: 507 out of 6070; (8.352553542009884)
RoBERTa results, first prediction correct: 679 out of 6070; (11.186161449752882) 
T5 results, first prediction correct: 600 out of 6070; (9.884678747940692)


In [26]:
bert_any_correct = bert_results.groupby('ids').any().correct.sum()
roberta_any_correct = roberta_results.groupby('ids').any().correct.sum()
t5_any_correct = t5_results.groupby('ids').any().correct.sum()

print(
    f"Bert results, any prediction correct: {bert_any_correct} out of 6070; ({bert_any_correct / 60.70})")
print(
    f"RoBERTa results, any prediction correct: {roberta_any_correct} out of 6070; ({roberta_any_correct/60.70}) ")
print(
    f"T5 results, any prediction correct: {t5_any_correct} out of 6070; ({t5_any_correct/60.70})")

Bert results, any prediction correct: 795 out of 6070; (13.097199341021415)
RoBERTa results, any prediction correct: 1052 out of 6070; (17.331136738056014) 
T5 results, any prediction correct: 891 out of 6070; (14.678747940691927)


In [32]:
# clozes_df = pd.read_json('pickles/clozes_20210807_165700.json')

In [86]:
clozes_df = pd.read_json('pickles/clozes_with_false_20210825.json')

In [87]:
all_data = []
for _, row in clozes_df.iterrows():
    data = row.data
    [all_data.append(x) for x in data if x not in all_data]

In [88]:
len(all_data)

912

In [103]:
import os
local_data = os.listdir('datasets/')
local_data = ['/'+x[:-4].replace('_', '/') for x in local_data]

missing = os.listdir('datasets/missing/')
missing = ['/'+x[:-4].replace('_', '/') for x in missing]

In [108]:
len(to_be_moved)

198

In [106]:
to_be_moved = [x for x in missing if x not in local_data]

In [112]:
import shutil
[shutil.move('datasets/missing/' + x.replace('/', '_')[1:] + '.xls', 'datasets/move') for x in to_be_moved]

['datasets/move/peoplepopulationandcommunity_birthsdeathsandmarriages_deaths_datasets_standardofproofsuicidedata.xls',
 'datasets/move/peoplepopulationandcommunity_crimeandjustice_datasets_coronavirusandcrimeinenglandandwalesappendixtables.xls',
 'datasets/move/peoplepopulationandcommunity_birthsdeathsandmarriages_deaths_datasets_theimpactofusingthenewdefinitionofalcoholspecificdeathsmaindatatables.xls',
 'datasets/move/employmentandlabourmarket_peoplenotinwork_unemployment_datasets_childrenbythecombinedeconomicactivitystatusofhouseholdmembersbylocalauthoritytablec1la.xls',
 'datasets/move/peoplepopulationandcommunity_crimeandjustice_datasets_crimeinenglandandwalesquarterlydatatables.xls',
 'datasets/move/peoplepopulationandcommunity_crimeandjustice_datasets_childabuseandthecriminaljusticesystemappendixtables.xls',
 'datasets/move/peoplepopulationandcommunity_birthsdeathsandmarriages_deaths_datasets_monthlymortalityanalysisenglandandwales.xls',
 'datasets/move/peoplepopulationandcommun

In [90]:
missing = [x for x in all_data if x not in local_data]

In [101]:
idx_of_missing = []
unusable_clozes = []
bulletins_of_missing = []
for idx, row in clozes_df.iterrows():
    data = row.data
    if any(df in missing for df in data):
        idx_of_missing.append(idx)
        bulletins_of_missing.append(row.bulletin)
    if all(df in missing for df in data):
        unusable_clozes.append(idx)

bulletins_of_missing = np.unique(bulletins_of_missing)

np.savetxt('secondary/bulletins_with_missing_dfs.txt', bulletins_of_missing, fmt = '%s', delimiter = ' ')

In [82]:
clozes_df

,ids,cloze_text,source_text,answer_text,answer_type,data
0,ccf75a2c157eacc5253f0e8a55b1ded9f5386d58,"In<mask>, approximate gross value added at bas...","In 2019, approximate gross value added at basi...",2019,DATE,[/businessindustryandtrade/business/businessse...
1,fc3346f2545ca5059bdb5150b998300c5001bea5,"In 2019, approximate gross value added at basi...","In 2019, approximate gross value added at basi...","£1,313.9 billion",MONEY,[/businessindustryandtrade/business/businessse...
2,81aa32270d547a368e7b806646c72069ba4fac43,"In 2019, approximate gross value added at basi...","In 2019, approximate gross value added at basi...",£42.8 billion,MONEY,[/businessindustryandtrade/business/businessse...
3,f308eb661ad1d6c8021cf9e22d8e8c7d9c5ff50e,"In 2019, approximate gross value added at basi...","In 2019, approximate gross value added at basi...",3.4%,PERCENT,[/businessindustryandtrade/business/businessse...
4,0a0bcf4b990f1926bc87fa662badadaf7e23d5fa,"In 2019, approximate gross value added at basi...","In 2019, approximate gross value added at basi...",2018,DATE,[/businessindustryandtrade/business/businessse...
...,...,...,...,...,...,...
4085,58116a07356fb4b001d48629d7864909c03fab79,Children who reported “low” satisfaction with ...,Children who reported “low” satisfaction with ...,28.3%,PERCENT,[/peoplepopulationandcommunity/wellbeing/datas...
4086,9902578d5ee75716bec570af2ae7ca8c378f039a,Children who reported “low” satisfaction with ...,Children who reported “low” satisfaction with ...,about 10%,PERCENT,[/peoplepopulationandcommunity/wellbeing/datas...
4087,2124dac9b26f153e90758d9f9f95d51c7aaeebef,9.8% of young people said that they were “ofte...,9.8% of young people said that they were “ofte...,9.8%,PERCENT,[/peoplepopulationandcommunity/wellbeing/datas...
4088,059317f10fe57b868e10900dd413e48a852ed0b3,Those reporting no long-term illness or disabi...,Those reporting no long-term illness or disabi...,44.8%,PERCENT,[/peoplepopulationandcommunity/wellbeing/datas...


['...']